In [1]:
import torch

In [2]:
import pandas as pd

In [10]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/11.6 MB 9.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/11.6 MB 9.5 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/11.6 MB 11.1 MB/s eta 0:00:01
   ------ --------------------------------- 1.9/11.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/11.6 MB 11.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.9/11.6 MB 10.8 MB/s eta 0:00:01
   ----------- ---------------------------- 3.4/11.6 MB 10.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.6 MB 10.8 MB/s eta 0:00:01
   ------------- -------------------------- 4.0/11.6 MB 9.6 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 9.3 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.6 MB 8.8 MB/s eta 0:00:01
   --------------- ------------------------ 4.6/11.6 MB 8.3 MB/s eta 0:00:01


In [3]:
import os
import shutil

import cv2
import numpy as np
from pycocotools.coco import COCO
from PIL import Image
import matplotlib.pyplot as plt

In [4]:

!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

C:\Users\sridh\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.
C:\Users\sridh\anaconda3\envs\pytorch_env\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-8 Python-3.9.19 torch-2.4.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [6]:
coco_images_path = r'C:\Users\sridh\Downloads\val2017\val2017'
coco_annotations_path = r"C:\Users\sridh\Downloads\annotations_trainval2017\annotations\instances_val2017.json"
classes_dir = 'classes'

output_folder='fp/fn analysis'

In [7]:
import json

# Load the JSON file
with open(r'C:\Users\sridh\Downloads\annotations_trainval2017\annotations\instances_val2017.json', 'r') as f:
    data = json.load(f)


In [8]:
# Extract essential components
images = [
    {
        "id": image["id"],
        "file_name": image["file_name"],
        "height": image["height"],
        "width": image["width"]
    }
    for image in data.get('images', [])
]

annotations = [
    {
        "id": annotation["id"],
        "image_id": annotation["image_id"],
        "category_id": annotation["category_id"],
        "bbox": annotation["bbox"],
        "area": annotation["area"],
        "iscrowd": annotation["iscrowd"]
    }
    for annotation in data.get('annotations', [])
]

categories = [
    {
        "id": category["id"],
        "name": category["name"]
    }
    for category in data.get('categories', [])
]

# Combine filtered data
clean_data = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# Save the clean JSON to a new file
with open('clean_annotations.json', 'w') as f:
    json.dump(clean_data, f, indent=4)


In [9]:
clean_data['images'][:5]

[{'id': 397133, 'file_name': '000000397133.jpg', 'height': 427, 'width': 640},
 {'id': 37777, 'file_name': '000000037777.jpg', 'height': 230, 'width': 352},
 {'id': 252219, 'file_name': '000000252219.jpg', 'height': 428, 'width': 640},
 {'id': 87038, 'file_name': '000000087038.jpg', 'height': 480, 'width': 640},
 {'id': 174482, 'file_name': '000000174482.jpg', 'height': 388, 'width': 640}]

In [10]:
clean_data['categories'][:5]

[{'id': 1, 'name': 'person'},
 {'id': 2, 'name': 'bicycle'},
 {'id': 3, 'name': 'car'},
 {'id': 4, 'name': 'motorcycle'},
 {'id': 5, 'name': 'airplane'}]

In [11]:
selected_classes=[
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]


In [12]:
for cls in selected_classes:
    os.makedirs(os.path.join(classes_dir, cls, 'fp'), exist_ok=True)
    os.makedirs(os.path.join(classes_dir, cls, 'fn'), exist_ok=True)



In [21]:
coco = COCO(coco_annotations_path)

loading annotations into memory...
Done (t=0.94s)
creating index...
index created!


In [13]:
def coco_to_xyxy(box):
    x, y, width, height = box
    return [x, y, x + width, y + height]

In [14]:
def draw_bounding_boxes(img, boxes, color, label=None):
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if label:
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

In [15]:
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


In [16]:
confidence_threshold = 0.4
iou_threshold = 0.5

In [58]:
import cv2
import os

def combine_images(gt_img_path, pred_img_path, output_path):
    # Read the ground truth and prediction images
    gt_img = cv2.imread(gt_img_path)
    pred_img = cv2.imread(pred_img_path)

    # Check if both images are loaded correctly
    if gt_img is None:
        print(f"Failed to read ground truth image: {gt_img_path}")
        return
    if pred_img is None:
        print(f"Failed to read prediction image: {pred_img_path}")
        return

    # Resize images to the same height for side-by-side concatenation
    height = max(gt_img.shape[0], pred_img.shape[0])
    gt_img_resized = cv2.resize(gt_img, (int(gt_img.shape[1] * (height / gt_img.shape[0])), height))
    pred_img_resized = cv2.resize(pred_img, (int(pred_img.shape[1] * (height / pred_img.shape[0])), height))

    # Concatenate images side by side
    combined_img = cv2.hconcat([gt_img_resized, pred_img_resized])

    # Save the combined image
    cv2.imwrite(output_path, combined_img)

def process_image(img_id):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_path, img_info['file_name'])

    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        return

    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
        return

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    # Print a sample annotation to inspect its structure
    if anns:
        print("Sample annotation:", anns[0])

    gt_boxes = {}
    for ann in anns:
        try:
            cls_name = coco.loadCats(ann['category_id'])[0]['name']
            if cls_name in selected_class:
                gt_boxes[cls_name] = coco_to_xyxy(ann['bbox'])
        except KeyError as e:
            print(f"KeyError: {e} in annotation: {ann}")

    results = model(img_path)
    predictions = results.xyxy[0].numpy()

    pred_boxes = {}
    for pred in predictions:
        pred_cls = model.names[int(pred[5])]
        pred_confidence = pred[4]
        pred_box = [int(pred[i]) for i in range(4)]
        if pred_confidence > confidence_threshold:
            pred_boxes[pred_cls] = pred_box

    false_positives = {}
    false_negatives = {}

    for gt_cls, gt_box in gt_boxes.items():
        if gt_cls not in pred_boxes:
            false_negatives[gt_cls] = (img_path, gt_box)
        else:
            pred_box = pred_boxes[gt_cls]
            iou = calculate_iou(gt_box, pred_box)
            if iou <= iou_threshold:
                false_negatives[gt_cls] = (img_path, gt_box)

    for pred_cls, pred_box in pred_boxes.items():
        if pred_cls not in gt_boxes:
            false_positives[pred_cls] = (img_path, pred_box)

    # Prepare paths for saving combined images
    for cls in selected_class:
        if cls in false_positives:
            img_path, fp_box = false_positives[cls]
            img = cv2.imread(img_path)
            draw_bounding_boxes(img, [fp_box], (0, 0, 255), f'{cls} FP')
            fp_save_path = os.path.join(classes_dir, cls, 'fp', os.path.basename(img_path))
            os.makedirs(os.path.dirname(fp_save_path), exist_ok=True)
            cv2.imwrite(fp_save_path, img)

        if cls in false_negatives:
            img_path, fn_box = false_negatives[cls]
            img = cv2.imread(img_path)
            draw_bounding_boxes(img, [fn_box], (255, 0, 0), f'{cls} FN')
            fn_save_path = os.path.join(classes_dir, cls, 'fn', os.path.basename(img_path))
            os.makedirs(os.path.dirname(fn_save_path), exist_ok=True)
            cv2.imwrite(fn_save_path, img)

    # Combine ground truth and prediction images
    for cls in selected_class:
        gt_img_path = os.path.join(classes_dir, cls, 'fn', os.path.basename(img_path))
        pred_img_path = os.path.join(classes_dir, cls, 'fp', os.path.basename(img_path))
        combined_img_path = os.path.join(output_folder, f'{os.path.basename(img_path)}')

        if os.path.exists(gt_img_path) and os.path.exists(pred_img_path):
            combine_images(gt_img_path, pred_img_path, combined_img_path)

print("Process completed.")

# Run the function for a specific image ID
img_id = 397133
process_image(img_id)


Process completed.
Processing image: C:\Users\sridh\Downloads\val2017\val2017\000000397133.jpg
Sample annotation: {'segmentation': [[224.24, 297.18, 228.29, 297.18, 234.91, 298.29, 243.0, 297.55, 249.25, 296.45, 252.19, 294.98, 256.61, 292.4, 254.4, 264.08, 251.83, 262.61, 241.53, 260.04, 235.27, 259.67, 230.49, 259.67, 233.44, 255.25, 237.48, 250.47, 237.85, 243.85, 237.11, 240.54, 234.17, 242.01, 228.65, 249.37, 224.24, 255.62, 220.93, 262.61, 218.36, 267.39, 217.62, 268.5, 218.72, 295.71, 225.34, 297.55]], 'area': 1481.3806499999994, 'iscrowd': 0, 'image_id': 397133, 'bbox': [217.62, 240.54, 38.99, 57.75], 'category_id': 44, 'id': 82445}


C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [17]:
def draw_bounding_boxes(img, boxes, color, label):
    """
    Draw bounding boxes on an image.
    """
    for box in boxes:
        if len(box) == 4:
            x1, y1, x2, y2 = box
            # Ensure coordinates are integers
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            if label:
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)



In [18]:
def draw_annotations(img, boxes, colors, labels):
    """
    Draw multiple annotations on an image.
    """
    for (box, color, label) in zip(boxes, colors, labels):
        draw_bounding_boxes(img, [box], color, label)



In [19]:
def combine_images_side_by_side(img1, img2):
    """
    Combine two images side by side.
    """
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    
    if h1 != h2:
        # Resize images to have the same height
        img1 = cv2.resize(img1, (w1 * h2 // h1, h2))
    
    combined_img = cv2.hconcat([img1, img2])
    return combined_img


In [70]:
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_path, img_info['file_name'])

    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
        continue

    img_width, img_height = img.shape[1], img.shape[0]

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    gt_boxes = []
    for ann in anns:
        cls_name = coco.loadCats(ann['category_id'])[0]['name']
        if cls_name in selected_classes:
            gt_boxes.append((cls_name, coco_to_xyxy(ann['bbox'])))

    results = model(img_path)
    predictions = results.xyxy[0].numpy()

    pred_boxes = []
    for pred in predictions:
        pred_cls = model.names[int(pred[5])]
        pred_confidence = pred[4]
        pred_box = [int(pred[i]) for i in range(4)]
        if pred_confidence > confidence_threshold:
            pred_boxes.append((pred_cls, pred_box))

    gt_img = img.copy()
    draw_annotations(gt_img, [box for _, box in gt_boxes], [(0, 255, 0)] * len(gt_boxes), ['GT'] * len(gt_boxes))

    pred_img = img.copy()
    draw_annotations(pred_img, [box for _, box in pred_boxes], [(0, 0, 255)] * len(pred_boxes), ['Pred'] * len(pred_boxes))


    combined_img = combine_images_side_by_side(gt_img, pred_img)


    for pred_cls, pred_box in pred_boxes:
        matched = False
        for gt_cls, gt_box in gt_boxes:
            if pred_cls == gt_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
     
            save_path_fp = os.path.join(classes_dir, pred_cls, 'fp', os.path.basename(img_path))
            cv2.imwrite(save_path_fp, combined_img)
            break  

    for gt_cls, gt_box in gt_boxes:
        matched = False
        for pred_cls, pred_box in pred_boxes:
            if gt_cls == pred_cls:
                iou = calculate_iou(gt_box, pred_box)
                if iou > iou_threshold:
                    matched = True
                    break
        if not matched:
            # Save as false negative
            save_path_fn = os.path.join(classes_dir, gt_cls, 'fn', os.path.basename(img_path))
            cv2.imwrite(save_path_fn, combined_img)
            break  # Move to next image after saving false negative

print("Process completed.")

C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models

Process completed.


In [ ]:
# Example lists of true labels and predictions
y_true = []  # Populate with ground truth labels
y_pred = []  # Populate with predictions

# Assuming you have a method to get these values from your dataset and model
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_path, img_info['file_name'])

    # Get ground truth labels
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        cls_name = coco.loadCats(ann['category_id'])[0]['name']
        if cls_name in selected_classes:
            y_true.append(cls_name)

    # Get model predictions
    results = model(img_path)
    predictions = results.xyxy[0].numpy()
    for pred in predictions:
        pred_cls = model.names[int(pred[5])]
        pred_confidence = pred[4]  # Confidence score is usually at index 4
        if pred_confidence > confidence_threshold:
            y_pred.append(pred_cls)


C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sridh/.cache\torch\hub\ultralytics_yolov5_master\models